# Read in the data

In [ ]:
import pandas
import numpy
import re

data_files = [
    "ap_2010.csv",
    "class_size.csv",
    "demographics.csv",
    "graduation.csv",
    "hs_directory.csv",
    "sat_results.csv"
]

data = {}

for f in data_files:
    d = pandas.read_csv("schools/{0}".format(f))
    data[f.replace(".csv", "")] = d

# Read in the surveys

In [ ]:
all_survey = pandas.read_csv("schools/survey_all.txt", delimiter="\t", encoding='windows-1252')
d75_survey = pandas.read_csv("schools/survey_d75.txt", delimiter="\t", encoding='windows-1252')
survey = pandas.concat([all_survey, d75_survey], axis=0)

survey["DBN"] = survey["dbn"]

survey_fields = [
    "DBN", 
    "rr_s", 
    "rr_t", 
    "rr_p", 
    "N_s", 
    "N_t", 
    "N_p", 
    "saf_p_11", 
    "com_p_11", 
    "eng_p_11", 
    "aca_p_11", 
    "saf_t_11", 
    "com_t_11", 
    "eng_t_10", 
    "aca_t_11", 
    "saf_s_11", 
    "com_s_11", 
    "eng_s_11", 
    "aca_s_11", 
    "saf_tot_11", 
    "com_tot_11", 
    "eng_tot_11", 
    "aca_tot_11",
]
survey = survey.loc[:,survey_fields]
data["survey"] = survey

# Add DBN columns

In [ ]:
data["hs_directory"]["DBN"] = data["hs_directory"]["dbn"]

def pad_csd(num):
    string_representation = str(num)
    if len(string_representation) > 1:
        return string_representation
    else:
        return "0" + string_representation
    
data["class_size"]["padded_csd"] = data["class_size"]["CSD"].apply(pad_csd)
data["class_size"]["DBN"] = data["class_size"]["padded_csd"] + data["class_size"]["SCHOOL CODE"]

# Convert columns to numeric

In [ ]:
cols = ['SAT Math Avg. Score', 'SAT Critical Reading Avg. Score', 'SAT Writing Avg. Score']
for c in cols:
    data["sat_results"][c] = pandas.to_numeric(data["sat_results"][c], errors="coerce")

data['sat_results']['sat_score'] = data['sat_results'][cols[0]] + data['sat_results'][cols[1]] + data['sat_results'][cols[2]]

def find_lat(loc):
    coords = re.findall("\(.+, .+\)", loc)
    lat = coords[0].split(",")[0].replace("(", "")
    return lat

def find_lon(loc):
    coords = re.findall("\(.+, .+\)", loc)
    lon = coords[0].split(",")[1].replace(")", "").strip()
    return lon

data["hs_directory"]["lat"] = data["hs_directory"]["Location 1"].apply(find_lat)
data["hs_directory"]["lon"] = data["hs_directory"]["Location 1"].apply(find_lon)

data["hs_directory"]["lat"] = pandas.to_numeric(data["hs_directory"]["lat"], errors="coerce")
data["hs_directory"]["lon"] = pandas.to_numeric(data["hs_directory"]["lon"], errors="coerce")

# Condense datasets

In [ ]:
class_size = data["class_size"]
class_size = class_size[class_size["GRADE "] == "09-12"]
class_size = class_size[class_size["PROGRAM TYPE"] == "GEN ED"]

class_size = class_size.groupby("DBN").agg(numpy.mean)
class_size.reset_index(inplace=True)
data["class_size"] = class_size

data["demographics"] = data["demographics"][data["demographics"]["schoolyear"] == 20112012]

data["graduation"] = data["graduation"][data["graduation"]["Cohort"] == "2006"]
data["graduation"] = data["graduation"][data["graduation"]["Demographic"] == "Total Cohort"]

# Convert AP scores to numeric

In [ ]:
cols = ['AP Test Takers ', 'Total Exams Taken', 'Number of Exams with scores 3 4 or 5']

for col in cols:
    data["ap_2010"][col] = pandas.to_numeric(data["ap_2010"][col], errors="coerce")

# Combine the datasets

In [ ]:
combined = data["sat_results"]

combined = combined.merge(data["ap_2010"], on="DBN", how="left")
combined = combined.merge(data["graduation"], on="DBN", how="left")

to_merge = ["class_size", "demographics", "survey", "hs_directory"]

for m in to_merge:
    combined = combined.merge(data[m], on="DBN", how="inner")

combined = combined.fillna(combined.mean())
combined = combined.fillna(0)

# Add a school district column for mapping

In [ ]:
def get_first_two_chars(dbn):
    return dbn[0:2]

combined["school_dist"] = combined["DBN"].apply(get_first_two_chars)

# Find correlations

In [ ]:
correlations = combined.corr()
correlations = correlations["sat_score"]
print(correlations)

In [ ]:
%matplotlib inline
survey_correlations = correlations[survey_fields]
survey_correlations.plot.bar()

## Observations regarding correlations between SAT scores and survey-related data
* Schools with high number of student and parent respondents had higher SAT scores. Those are numbers and not rates and may just reflect bigger schools.
* The student response rate (rr_s) correlates pretty well sat_score. This isn't surprising.
* SAT scores were inversely related to the communication score based on parent responses. 
* Students' perception of the academic standard (aca_s) did correlate with sat_scores, although - interestingly - their parents' and teachers' assessment did not correlate.


In [ ]:
safety = combined[['sat_score','saf_t_11', 'saf_s_11']]
safety.plot.scatter('saf_s_11', 'sat_score')

## Observation about student perception of safety and SAT scores
There seem to be two clusters:
* SAT scores are relatively flat for the lower end of the 
satisfaction scores. As satisfaction increases, scores don't go up.
* However, there is a second clustering at the higher end of the 
satisfaction scores. There, the scores suddenly skyrocket.

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

districts = combined.groupby('school_dist').agg(numpy.mean)
districts.reset_index(inplace=True)

m = Basemap(
    projection='merc', 
    llcrnrlat=40.496044, 
    urcrnrlat=40.915256, 
    llcrnrlon=-74.255735, 
    urcrnrlon=-73.700272,
    resolution='i'
)

m.drawmapboundary(fill_color='#85A6D9')
m.drawcoastlines(color='#6D5F47', linewidth=.4)
m.drawrivers(color='#6D5F47', linewidth=.4)
m.fillcontinents(color='white',lake_color='#85A6D9')

longitudes = districts["lon"].tolist()
latitudes = districts["lat"].tolist()

m.scatter(longitudes, latitudes, s=50, zorder=2, latlon=True, 
          c=districts['saf_s_11'], cmap = 'summer')


# Observation about safety scores by district
There isn't a very clear pattern that emerges.
I don't agree with the conlclusion provided in the solution, "It looks like Upper Manhattan and parts of Queens and the Bronx tend to have lower safety scores, whereas Brooklyn has high safety scores." Brooklyn seems to have as many "unsafe" districts as Queens.

In [ ]:
demographics = correlations[['white_per', 'asian_per', 'black_per', 'hispanic_per']]
demographics.plot.bar()

SAT scores correlate positively with percentage of white and percentage of asians in a school; while being inversely related to percentage of blacks and hispanics. I'm a bit surprised that the negative correlations aren't as strong as the positive correlations. 

In [ ]:
combined.plot.scatter('hispanic_per','sat_score')

It seems to be a pretty straightforward negative correlation. Percentage of Latinos goes up, scores go down.

In [ ]:
combined_filtered = combined[combined['hispanic_per']>95]
combined_filtered.reset_index(inplace=True)
print(combined_filtered['SCHOOL NAME'])

It seems that these schools serve international students who are learning English.

In [ ]:
low_latino = combined[(combined['hispanic_per']<10) & (combined['sat_score']>1800)]
low_latino.reset_index(inplace=True)
print(low_latino['SCHOOL NAME'])

These are magnet schools.

In [ ]:
high_sat = combined[(combined['sat_score']>1800)]
high_sat.reset_index(inplace=True)
print(high_sat[['SCHOOL NAME','sat_score','white_per','hispanic_per']])

In [ ]:
gender_corr = correlations[['male_per', 'female_per']]
gender_corr.plot.bar()

Girls do better than boys on the SATs, but the correlation isn't strong.

In [ ]:
combined.plot.scatter('female_per','sat_score')

The scatter plot reveals that there really isn't any consistent trend. The correlation that showed up was only about 0.1 and doesn't seem to reveal any real correlation.

In [ ]:
high_fem = combined[(combined['female_per']>60) & (combined['sat_score']>1700)]
high_fem.reset_index(inplace=True)
print(high_fem['SCHOOL NAME'])

These are specialty magnet schools.

In [ ]:
combined['ap_per']=combined['AP Test Takers ']/combined['total_enrollment']
combined.plot.scatter(x='ap_per', y='sat_score')